In [2]:
import sys
sys.path.append("../MMPF")
sys.path.append("../scripts")

import data
import models



%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
dataset = "acsincome"
n_groups = 8
fold = 0
(
        X_train,
        A_train,
        Y_train,
        X_val,
        A_val,
        Y_val,
        X_test,
        A_test,
        Y_test,
) = data.get_strat_split(dataset, n_groups, 20)

In [ ]:
m = models.MinimaxPareto(n_iterations = 10, max_iter = 10)
m.fit(X_train, Y_train, A_train)